<a href="https://colab.research.google.com/github/PrithvirajKhelkar/intent-classification-shallow-ml/blob/main/intent_classification_shallow_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import required libraries

In [7]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

## loads the train and test datasets from [here](https://www.kaggle.com/datasets/hassanamin/atis-airlinetravelinformationsystem)